In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


In [ ]:
#크롤링 정보를 저장할 데이터프레임 선언
data = pd.DataFrame(columns =['스테이션명','위치','상태정보','위도','경도'])


#크롤링 할 타슈 사이트 주소
url_tashu ="http://www.tashu.or.kr/userStationAction.do?process=list"

#페이지 이동시 추가로 적어줘야되는 url
url_add ="&page="

#전체 페이지 수
total_page =27

#27 page에 대한 정보를 추출하는 for문

for index in range(1, total_page+1):

  if index == 1 :
    url = url_tashu# 첫 번째 페이지
  else:
    url = url_tashu + url_add + str(index) # 페이지 정보 추가

    #url에서 html 추출
  html = urlopen(url)

    # html을 파싱하여 데이터로 변환
  bsObject = BeautifulSoup(html,"html.parser", from_encoding='utf-8')

    # history_table이라는 class를 가진 table만 추출
  table = bsObject.find_all('table', {'class','history_table'})

    # table 내에서 tr 추출 
  tr = table[0].find_all('tr')

    # 첫 tr은 contents 가 아니라 데이터 정보를 의미하기 때문에 제외 [1:tr수까지]
  tr = tr[1:len(tr)]

    #한 페이지의 정보를 크롤링
  for index_tr in range(0, len(tr)):
    td = tr[index_tr].find_all('td')

      #장소
    station = td[0].text.split('.')[1]# 1. 무역전시관입구(택시승강장)

      #위치
    location = td[1].text

      #상태
    condition = td[2].text

      #좌표추출
    geo = td[3].a.attrs['href'].split('(')[1]  #'36.374389','127.392375');',추출

      #위도,경도
    geo = re.sub('[;)]','',geo) #36.374389','127.392375');' ->'36.374389','127.392375'
    detail_location = geo.replace("'","")#작은 따옴표 삭제

      #위도 Latitude
    lat = detail_location.split(',')[0]

      #경도 Longitude
    lon = detail_location.split(',')[1]

      # data에 크롤링 결과 저장
    data = data.append({'스테이션명':station,
                          '위치':location,
                          '상태정보':condition,
                          '위도':lat,
                          '경도':lon},
                         ignore_index =True)
    print('Complete of ' + str(index_tr+1) + ' infomation with ' +str(index) + 'page')

  #Crawling 막히는 것 방지
  time.sleep(1)

print('-----------------------------------------------------------------------------------------')
print(data)

Complete of 1 infomation with 1page
Complete of 2 infomation with 1page
Complete of 3 infomation with 1page
Complete of 4 infomation with 1page
Complete of 5 infomation with 1page
Complete of 6 infomation with 1page
Complete of 7 infomation with 1page
Complete of 8 infomation with 1page
Complete of 9 infomation with 1page
Complete of 10 infomation with 1page
Complete of 1 infomation with 2page
Complete of 2 infomation with 2page
Complete of 3 infomation with 2page
Complete of 4 infomation with 2page
Complete of 5 infomation with 2page
Complete of 6 infomation with 2page
Complete of 7 infomation with 2page
Complete of 8 infomation with 2page
Complete of 9 infomation with 2page
Complete of 10 infomation with 2page
Complete of 1 infomation with 3page
Complete of 2 infomation with 3page
Complete of 3 infomation with 3page
Complete of 4 infomation with 3page
Complete of 5 infomation with 3page
Complete of 6 infomation with 3page
Complete of 7 infomation with 3page
Complete of 8 infomation w

**크롤링 저장하기**

In [ ]:
data.to_csv('TASHU.csv', encoding='utf-8-sig')